In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


 ## Bước 1: Đọc và chuẩn bị dữ liệu

In [2]:
import pandas as pd
train_df = pd.read_csv("/content/drive/MyDrive/train_data_attrition_scaling.csv")
test_df = pd.read_csv("/content/drive/MyDrive/test_data_attrition_scaling.csv")

In [3]:
# Xác định cột mục tiêu
target = 'Attrition'

 ## Bước 2: Chia dữ liệu thành 2 tập train-test

In [4]:
from sklearn.model_selection import train_test_split

# Tách features (X) và target (y) cho tập huấn luyện
X_train = train_df.drop(columns=[target])
y_train = train_df[target]

# Tách features (X) và target (y) cho tập kiểm tra
X_test = test_df.drop(columns=[target])
y_test = test_df[target]

print("Kích thước tập huấn luyện (X_train, y_train):", X_train.shape, y_train.shape)
print("Kích thước tập kiểm tra (X_test, y_test):", X_test.shape, y_test.shape)

Kích thước tập huấn luyện (X_train, y_train): (1574, 34) (1574,)
Kích thước tập kiểm tra (X_test, y_test): (294, 34) (294,)


In [7]:

!pip install optuna

## Bước 3: Định nghĩa hàm tối ưu hóa Optuna

In [8]:
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 20),
        'min_samples_split': trial.suggest_int('min_samples_split', 2, 10),
        'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2']),
        'criterion': trial.suggest_categorical('criterion', ['gini', 'entropy'])
    }
    model = RandomForestClassifier(**params, random_state=42)
    score = cross_val_score(model, X_train, y_train, cv=5, scoring='f1').mean()  # Dùng F1 vì dữ liệu imbalance
    return score

## Bước 4: Chạy Optuna để tìm siêu tham số tối ưu

In [10]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)
print("Best params:", study.best_params)

[I 2025-06-12 20:30:00,151] A new study created in memory with name: no-name-86aedb16-5512-4b12-ab5f-77c82ffb3c75
[I 2025-06-12 20:30:05,508] Trial 0 finished with value: 0.8596491595639912 and parameters: {'n_estimators': 112, 'max_depth': 18, 'min_samples_split': 6, 'max_features': 'log2', 'criterion': 'entropy'}. Best is trial 0 with value: 0.8596491595639912.
[I 2025-06-12 20:30:11,656] Trial 1 finished with value: 0.8539500491814913 and parameters: {'n_estimators': 240, 'max_depth': 8, 'min_samples_split': 6, 'max_features': 'log2', 'criterion': 'entropy'}. Best is trial 0 with value: 0.8596491595639912.
[I 2025-06-12 20:30:14,839] Trial 2 finished with value: 0.8546093075974335 and parameters: {'n_estimators': 115, 'max_depth': 13, 'min_samples_split': 9, 'max_features': 'log2', 'criterion': 'entropy'}. Best is trial 0 with value: 0.8596491595639912.
[I 2025-06-12 20:30:17,094] Trial 3 finished with value: 0.8633167768802237 and parameters: {'n_estimators': 118, 'max_depth': 19, 

Best params: {'n_estimators': 183, 'max_depth': 18, 'min_samples_split': 3, 'max_features': 'sqrt', 'criterion': 'gini'}


## Bước 5: Huấn luyện mô hình cuối cùng với siêu tham số tối ưu

In [11]:
best_params = study.best_params
final_model = RandomForestClassifier(**best_params, random_state=42)
final_model.fit(X_train, y_train)

RandomForestClassifier(max_depth=18, min_samples_split=3, n_estimators=183,
                       random_state=42)

## Bước 6: Đánh giá mô hình

In [12]:
from sklearn.metrics import classification_report
y_pred = final_model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.90      0.92      0.91       255
           1       0.40      0.36      0.38        39

    accuracy                           0.84       294
   macro avg       0.65      0.64      0.64       294
weighted avg       0.84      0.84      0.84       294



## Bước 7: Lưu mô hình

In [13]:
import joblib
joblib.dump(final_model, 'attrition_model.pkl')

['attrition_model.pkl']

In [14]:
# Dự đoán nhãn lớp (0: No Attrition, 1: Attrition)
y_pred = final_model.predict(X_test)
print("Dự đoán nhãn:", y_pred[:10])  # Hiển thị 10 dự đoán đầu tiên

Dự đoán nhãn: [0 0 0 0 0 0 0 0 0 0]


In [15]:
# Dự đoán xác suất cho từng lớp (cột 0: P(No Attrition), cột 1: P(Attrition))
y_proba = final_model.predict_proba(X_test)
print("Xác suất dự đoán:\n", y_proba[:5])  # Hiển thị 5 dòng đầu

Xác suất dự đoán:
 [[0.82586521 0.17413479]
 [0.76083789 0.23916211]
 [0.58779599 0.41220401]
 [0.76047359 0.23952641]
 [0.74954463 0.25045537]]


In [16]:
# Tạo DataFrame từ kết quả
results_df = pd.DataFrame({
    'Actual': y_test,                # Giá trị thực tế
    'Predicted': y_pred,             # Dự đoán nhãn
    'Probability_No_Attrition': y_proba[:, 0],  # Xác suất lớp 0
    'Probability_Attrition': y_proba[:, 1]      # Xác suất lớp 1
})

# Lưu vào file CSV
#results_df.to_csv('attrition_predictions.csv', index=False)
#print("Đã lưu kết quả vào 'attrition_predictions.csv'")

In [17]:
results_df

,Actual,Predicted,Probability_No_Attrition,Probability_Attrition
0,0,0,0.825865,0.174135
1,0,0,0.760838,0.239162
2,1,0,0.587796,0.412204
3,0,0,0.760474,0.239526
4,0,0,0.749545,0.250455
...,...,...,...,...
289,0,0,0.839800,0.160200
290,0,0,0.671220,0.328780
291,0,0,0.676685,0.323315
292,0,0,0.571494,0.428506
